In [3]:
import pandas as pd

import spacy
from spacy import displacy

from spacy.matcher import Matcher
from spacy.matcher import DependencyMatcher
from spacy.tokens import Token

Token.set_extension("ignore", default=False, force=True)

nlp = spacy.load("en_core_web_trf")

In [4]:
sentences = pd.read_excel('../data/demo_data_real.xlsx')
sentence = sentences.iloc[13]['Original Text EN']
sentences

,ID,Bereinigter Text,Tags/Kategorie,Eingereicht von,Original Text,Original Text EN
0,1,Die Tabelle “Subscriptions” im Schema “CRM” en...,"DWH, SQL",NaN,"Hey, du kannst die Tabelle “Subscriptions” im ...","Hey, you can use the ""Subscriptions"" table in ..."
1,2,Maximal 3 Urlaubstage mit ins nächste Jahr neh...,HR,NaN,"Hallo Marcel, hier noch eine kurze Info für di...","Hello Marcel, here is a short info for you: Yo..."
2,3,CR bedeutet Conversion Rate.,"Allgemeine Definitionen, CRM",NaN,"Hi Marcel, CR bedeutet in unserem Kontext Conv...","Hi Marcel, CR in our context means conversion ..."
3,4,AE bedeutet Account Executive.,"Allgemeine Definitionen, CRM",NaN,"AE = Account Executive, also Sales Mitarbeiter...","AE = Account Executive, i.e. Sales employees w..."
4,5,Mit “Abrechnungsdatum” ist das Attribut “Purch...,"Allgemeine Definitionen, DWH",NaN,"Hi Marcel, ja genau mit “Abrechnungsdatum” ist...","Hi Marcel, yes exactly with ""Billing Date"" the..."
5,6,Die gebuchten Addons sind in der Tabelle “Subs...,"DWH, SQL",NaN,"Hey Marcel, die gebuchten Addons findest du in...","Hey Marcel, you can find the booked addons in ..."
6,7,"""Deactivated Date” steht für den Zeitpunkt, wa...","Allgemeine Definitionen, CRM",NaN,"Hi, das Attribut “Deactivated Date” steht für ...","Hi, the attribute ""Deactivated Date"" stands fo..."
7,8,"Die wichtigsten Tabellen sind “Owners”, “Subsc...","CRM, DWH",NaN,"Hey Marcel, erstmal noch eine kurze Info für d...","Hey Marcel, first of all a short info for you:..."
8,9,Mit dem Datenintegrationstool Stitch können wi...,DWH,NaN,"Hey Marcel, ich weiß nicht, ob du das Tool Sti...","Hey Marcel, I don't know if you know about the..."
9,10,Als Werkstudent ist der Abrechnungszeitraum fü...,HR,NaN,"Hi Marcel, eine kurze Info noch für dich: Als ...","Hi Marcel, a short info for you: As a working ..."


# General Preprocessing

In [132]:
sentences.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=[" "," "], regex=True, inplace=True)

In [133]:
sentences

,ID,Bereinigter Text,Tags/Kategorie,Eingereicht von,Original Text,Original Text EN
0,1,Die Tabelle “Subscriptions” im Schema “CRM” en...,"DWH, SQL",NaN,"Hey, du kannst die Tabelle “Subscriptions” im ...","Hey, you can use the ""Subscriptions"" table in ..."
1,2,Maximal 3 Urlaubstage mit ins nächste Jahr neh...,HR,NaN,"Hallo Marcel, hier noch eine kurze Info für di...","Hello Marcel, here is a short info for you: Yo..."
2,3,CR bedeutet Conversion Rate.,"Allgemeine Definitionen, CRM",NaN,"Hi Marcel, CR bedeutet in unserem Kontext Conv...","Hi Marcel, CR in our context means conversion ..."
3,4,AE bedeutet Account Executive.,"Allgemeine Definitionen, CRM",NaN,"AE = Account Executive, also Sales Mitarbeiter...","AE = Account Executive, i.e. Sales employees w..."
4,5,Mit “Abrechnungsdatum” ist das Attribut “Purch...,"Allgemeine Definitionen, DWH",NaN,"Hi Marcel, ja genau mit “Abrechnungsdatum” ist...","Hi Marcel, yes exactly with ""Billing Date"" the..."
5,6,Die gebuchten Addons sind in der Tabelle “Subs...,"DWH, SQL",NaN,"Hey Marcel, die gebuchten Addons findest du in...","Hey Marcel, you can find the booked addons in ..."
6,7,"""Deactivated Date” steht für den Zeitpunkt, wa...","Allgemeine Definitionen, CRM",NaN,"Hi, das Attribut “Deactivated Date” steht für ...","Hi, the attribute ""Deactivated Date"" stands fo..."
7,8,"Die wichtigsten Tabellen sind “Owners”, “Subsc...","CRM, DWH",NaN,"Hey Marcel, erstmal noch eine kurze Info für d...","Hey Marcel, first of all a short info for you:..."
8,9,Mit dem Datenintegrationstool Stitch können wi...,DWH,NaN,"Hey Marcel, ich weiß nicht, ob du das Tool Sti...","Hey Marcel, I don't know if you know about the..."
9,10,Als Werkstudent ist der Abrechnungszeitraum fü...,HR,NaN,"Hi Marcel, eine kurze Info noch für dich: Als ...","Hi Marcel, a short info for you: As a working ..."


# Generate Rules

In [48]:
doc = nlp(sentence)

In [49]:
displacy.serve(doc, style='dep')

c:\Users\jdhau\anaconda3\lib\site-packages\spacy\displacy\__init__.py:103: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [5]:
def set_ignore(matcher, doc, id, matches):
    for _, start, end in matches:
        for tok in doc[start:end]:
            tok._.ignore = True

In [11]:
match_texts = Matcher(nlp.vocab)

In [12]:
sentence = sentences.iloc[13]['Original Text EN']
print(sentence)
doc = nlp(sentence)

Hi all, we are now authorized in d-bru-04 and p-bru-01 on Grafana. Entry best not via direct link, but via Rancher>Monitoring>Grafana Greetings Dominik


## Grußformeln (Ende)

In [13]:
pattern = [
    [{"POS": "ADJ"}, {"POS": "NOUN"}, {"POS": "PROPN"}],
    [{"POS": "NOUN"}, {"POS": "PROPN"}],
    ]
match_texts.add("Grußformeln", pattern, on_match=set_ignore)

In [42]:
# matches = Grußformelmatcher(doc)

# print(matches)

# for match_id, start, end in matches:
#     string_id = nlp.vocab.strings[match_id]  # Get string representation
#     span = doc[start:end]  # The matched span
#     print(match_id, string_id, start, end, span.text)

[(1981270878604495065, 32, 34)]
1981270878604495065 Grußformeln 32 34 Greetings Dominik


## Begrüßungsformeln (EN)

In [14]:


hello_synoym = ["hello", "hi", "greetings", "welcome", "hey", "olla", "hi-ya", "howdy"] #"good morning", "good evening", "good afternoon", 

pattern = [
    [{"LOWER": {"IN": hello_synoym}}, {"POS": "PROPN"}, {"IS_PUNCT": True}],  
    [{"LOWER": {"IN": hello_synoym}}, {"POS": "ADV"}, {"IS_PUNCT": True}],
    [{"LOWER": {"IN": hello_synoym}}, {"POS": "PRON"}, {"IS_PUNCT": True}],
]

match_texts.add("Begrusungformeln", pattern, on_match=set_ignore) 

In [55]:
# matches = match_texts(doc)

# print(matches)

# for match_id, start, end in matches:
#     string_id = nlp.vocab.strings[match_id]  # Get string representation
#     span = doc[start:end]  # The matched span
#     print(match_id, string_id, start, end, span.text)

[(1917771495493254596, 0, 3)]
1917771495493254596 Begrusungformeln 0 3 Hi all,


## Füllwortmatcher

In [56]:
# Füllwortmatcher = Matcher(nlp.vocab)

# pattern = [
#     [{"lower": "erstmal"}],
#     [{"lower": "erst"}, {"lower": "einmal"}],
#     [{"lower": "ja"}, {"lower": "genau"}],
#     [{"lower": "exakt"}],
#     [{"lower": "jawohl"}],
#     ]
# Füllwortmatcher.add("Füllwörter", pattern, on_match=set_ignore)



In [57]:
# matches = Füllwortmatcher(doc)

# print(matches)

# for match_id, start, end in matches:
#     string_id = nlp.vocab.strings[match_id]  # Get string representation
#     span = doc[start:end]  # The matched span
#     print(match_id, string_id, start, end, span.text)

[]


## Resultat

In [15]:
matches = match_texts(doc)

print(matches)

for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

[(1917771495493254596, 0, 3), (1981270878604495065, 32, 34)]
1917771495493254596 Begrusungformeln 0 3 Hi all,
1981270878604495065 Grußformeln 32 34 Greetings Dominik


In [19]:
toks = [tok.text + tok.whitespace_ for tok in doc if not tok._.ignore]
cleaned_text = "".join(toks)
cleaned_text = cleaned_text[0].upper() + cleaned_text[1:]
print(cleaned_text)

We are now authorized in d-bru-04 and p-bru-01 on Grafana. Entry best not via direct link, but via Rancher>Monitoring>Grafana 


# Multiprocessing

In [27]:
# docs = nlp.pipe(sentences['Original Text'], n_process=4)

# Augmentation

In [86]:
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas

Backtranslating does not work

In [99]:
# aug = naw.BackTranslationAug(
#     from_model_name='facebook/wmt19-en-de', 
#     to_model_name='facebook/wmt19-de-en'
# )

# back_translated = sentences["Original Text EN"].apply(lambda x: aug.augment(x)[0])

# print(back_translated)

# sentences_back_translated = sentences
# sentences_back_translated['Original Text EN'] = back_translated

# sentences_backtrans = sentences.append(sentences_back_translated)
# sentences.info()

0     Hey, you can use the table "Subscriptions" in ...
1     Hello Marcel, here is a short info for you: Yo...
2     Hello Marcel, in our context CR means conversi...
3     AE = Account Executive, i.e. sales staff who t...
4     Hello Marcel, yes, exactly "billing date" mean...
5     Hello Marcel, you can find the booked addons i...
6     Hello, the "Deactivated Date" attribute repres...
7     Hello Marcel, first a brief piece of informati...
8     Hi Marcel, I don't know if you still know the ...
9     Hello Marcel, a short piece of information for...
10    Hello, I have put together here: http: / / sew...
11    Hello everyone, I solved the problem with MLFl...
12    Hi Jonas, There is a secret in every namespace...
13    Hello everyone, we are now authorized in d-bru...
14    Hi Nico, have you already connected the DeepL ...
15    Hello, yes, we have MARA and MARC. In the tabl...
16    non-Tans = materials with material number (sta...
17    Hi Tatyana, I got to the entries via SE16 

C:\Users\jdhau\AppData\Local\Temp\ipykernel_17080\2319275326.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sentences_backtrans = sentences.append(sentences_back_translated)


Use Synonyms instead

In [134]:
sentences

,ID,Bereinigter Text,Tags/Kategorie,Eingereicht von,Original Text,Original Text EN
0,1,Die Tabelle “Subscriptions” im Schema “CRM” en...,"DWH, SQL",NaN,"Hey, du kannst die Tabelle “Subscriptions” im ...","Hey, you can use the ""Subscriptions"" table in ..."
1,2,Maximal 3 Urlaubstage mit ins nächste Jahr neh...,HR,NaN,"Hallo Marcel, hier noch eine kurze Info für di...","Hello Marcel, here is a short info for you: Yo..."
2,3,CR bedeutet Conversion Rate.,"Allgemeine Definitionen, CRM",NaN,"Hi Marcel, CR bedeutet in unserem Kontext Conv...","Hi Marcel, CR in our context means conversion ..."
3,4,AE bedeutet Account Executive.,"Allgemeine Definitionen, CRM",NaN,"AE = Account Executive, also Sales Mitarbeiter...","AE = Account Executive, i.e. Sales employees w..."
4,5,Mit “Abrechnungsdatum” ist das Attribut “Purch...,"Allgemeine Definitionen, DWH",NaN,"Hi Marcel, ja genau mit “Abrechnungsdatum” ist...","Hi Marcel, yes exactly with ""Billing Date"" the..."
5,6,Die gebuchten Addons sind in der Tabelle “Subs...,"DWH, SQL",NaN,"Hey Marcel, die gebuchten Addons findest du in...","Hey Marcel, you can find the booked addons in ..."
6,7,"""Deactivated Date” steht für den Zeitpunkt, wa...","Allgemeine Definitionen, CRM",NaN,"Hi, das Attribut “Deactivated Date” steht für ...","Hi, the attribute ""Deactivated Date"" stands fo..."
7,8,"Die wichtigsten Tabellen sind “Owners”, “Subsc...","CRM, DWH",NaN,"Hey Marcel, erstmal noch eine kurze Info für d...","Hey Marcel, first of all a short info for you:..."
8,9,Mit dem Datenintegrationstool Stitch können wi...,DWH,NaN,"Hey Marcel, ich weiß nicht, ob du das Tool Sti...","Hey Marcel, I don't know if you know about the..."
9,10,Als Werkstudent ist der Abrechnungszeitraum fü...,HR,NaN,"Hi Marcel, eine kurze Info noch für dich: Als ...","Hi Marcel, a short info for you: As a working ..."


In [135]:
aug = naw.SynonymAug(aug_src='wordnet', aug_max=2)

synonym_words = sentences["Original Text EN"].apply(lambda x: aug.augment(x,4))

sentences_synonym_words = sentences.copy()
sentences_synonym_words['Original Text EN'] = synonym_words
sentences_synonym_words = sentences_synonym_words.explode('Original Text EN')
sentences_synonym_words = sentences_synonym_words.reset_index()

sentences_synonym = sentences.append(sentences_synonym_words)

sentences_synonym

C:\Users\jdhau\AppData\Local\Temp\ipykernel_17080\1794860466.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sentences_synonym = sentences.append(sentences_synonym_words)


,ID,Bereinigter Text,Tags/Kategorie,Eingereicht von,Original Text,Original Text EN,index
0,1,Die Tabelle “Subscriptions” im Schema “CRM” en...,"DWH, SQL",NaN,"Hey, du kannst die Tabelle “Subscriptions” im ...","Hey, you can use the ""Subscriptions"" table in ...",NaN
1,2,Maximal 3 Urlaubstage mit ins nächste Jahr neh...,HR,NaN,"Hallo Marcel, hier noch eine kurze Info für di...","Hello Marcel, here is a short info for you: Yo...",NaN
2,3,CR bedeutet Conversion Rate.,"Allgemeine Definitionen, CRM",NaN,"Hi Marcel, CR bedeutet in unserem Kontext Conv...","Hi Marcel, CR in our context means conversion ...",NaN
3,4,AE bedeutet Account Executive.,"Allgemeine Definitionen, CRM",NaN,"AE = Account Executive, also Sales Mitarbeiter...","AE = Account Executive, i.e. Sales employees w...",NaN
4,5,Mit “Abrechnungsdatum” ist das Attribut “Purch...,"Allgemeine Definitionen, DWH",NaN,"Hi Marcel, ja genau mit “Abrechnungsdatum” ist...","Hi Marcel, yes exactly with ""Billing Date"" the...",NaN
...,...,...,...,...,...,...,...
103,26,Wie kann ich die Berechtigungen / Sap-User bea...,"HR, SAP",NaN,"Hallo Milagros, ich habe aktuell nur einen Nut...","Hi Milagros, I currently have only one substan...",25.0
104,27,Azure DevOps Lizenz. Wie war da der Prozess? ...,HR,NaN,"Hi Niklas, Stefan meinte, dass du für deinen P...","Hi Niklas, Stefan said that you had an Azure D...",26.0
105,27,Azure DevOps Lizenz. Wie war da der Prozess? ...,HR,NaN,"Hi Niklas, Stefan meinte, dass du für deinen P...","Hi Niklas, Stefan said that you had an Azure D...",26.0
106,27,Azure DevOps Lizenz. Wie war da der Prozess? ...,HR,NaN,"Hi Niklas, Stefan meinte, dass du für deinen P...","Hi Niklas, Stefan said that you had an Azure D...",26.0


Swap parts

In [137]:
aug = naw.RandomWordAug(action="swap", aug_max=2)

swapped_words = sentences_synonym["Original Text EN"].apply(lambda x: aug.augment(x,4))

sentences_swapped_words = sentences_synonym.copy()
sentences_swapped_words['Original Text EN'] = swapped_words
sentences_swapped_words = sentences_swapped_words.explode('Original Text EN')
sentences_swapped_words = sentences_swapped_words.reset_index()

sentences_synonym_swap = sentences_synonym.append(sentences_swapped_words)

sentences_synonym_swap

C:\Users\jdhau\AppData\Local\Temp\ipykernel_17080\2742183552.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sentences_synonym_swap = sentences_synonym.append(sentences_swapped_words)


,ID,Bereinigter Text,Tags/Kategorie,Eingereicht von,Original Text,Original Text EN,index,level_0
0,1,Die Tabelle “Subscriptions” im Schema “CRM” en...,"DWH, SQL",NaN,"Hey, du kannst die Tabelle “Subscriptions” im ...","Hey, you can use the ""Subscriptions"" table in ...",NaN,NaN
1,2,Maximal 3 Urlaubstage mit ins nächste Jahr neh...,HR,NaN,"Hallo Marcel, hier noch eine kurze Info für di...","Hello Marcel, here is a short info for you: Yo...",NaN,NaN
2,3,CR bedeutet Conversion Rate.,"Allgemeine Definitionen, CRM",NaN,"Hi Marcel, CR bedeutet in unserem Kontext Conv...","Hi Marcel, CR in our context means conversion ...",NaN,NaN
3,4,AE bedeutet Account Executive.,"Allgemeine Definitionen, CRM",NaN,"AE = Account Executive, also Sales Mitarbeiter...","AE = Account Executive, i.e. Sales employees w...",NaN,NaN
4,5,Mit “Abrechnungsdatum” ist das Attribut “Purch...,"Allgemeine Definitionen, DWH",NaN,"Hi Marcel, ja genau mit “Abrechnungsdatum” ist...","Hi Marcel, yes exactly with ""Billing Date"" the...",NaN,NaN
...,...,...,...,...,...,...,...,...
535,27,Azure DevOps Lizenz. Wie war da der Prozess? ...,HR,NaN,"Hi Niklas, Stefan meinte, dass du für deinen P...","Hi Niklas, Stefan said that you had an Azure D...",26.0,106.0
536,27,Azure DevOps Lizenz. Wie war da der Prozess? ...,HR,NaN,"Hi Niklas, Stefan meinte, dass du für deinen P...","Hi Niklas, Stefan said that had you an Azure D...",26.0,107.0
537,27,Azure DevOps Lizenz. Wie war da der Prozess? ...,HR,NaN,"Hi Niklas, Stefan meinte, dass du für deinen P...","Hi Niklas, Stefan said that had you an Azure D...",26.0,107.0
538,27,Azure DevOps Lizenz. Wie war da der Prozess? ...,HR,NaN,"Hi Niklas, Stefan meinte, dass du für deinen P...","Hi Niklas, Stefan said that you had an Azure D...",26.0,107.0


In [139]:
sentences_synonym_swap['Original Text EN'].nunique()

664

Languagemodel Magic

In [150]:
aug = naw.ContextualWordEmbsAug(
    model_path='roberta-base', action="substitute")

magic_words = sentences_synonym_swap["Original Text EN"].apply(lambda x: aug.augment(x,4))

sentences_magic_words = sentences_synonym_swap.copy()
sentences_magic_words['Original Text EN'] = magic_words
sentences_magic_words = sentences_magic_words.explode('Original Text EN')
sentences_magic_words = sentences_magic_words.drop(columns=['level_0'])
sentences_magic_words = sentences_magic_words.reset_index()

sentences_synonym_swap_magic = sentences_synonym_swap.append(sentences_magic_words)

sentences_synonym_swap_magic

,ID,Bereinigter Text,Tags/Kategorie,Eingereicht von,Original Text,Original Text EN,index,level_0
0,1,Die Tabelle “Subscriptions” im Schema “CRM” en...,"DWH, SQL",NaN,"Hey, du kannst die Tabelle “Subscriptions” im ...","Hey, you can use the ""Subscriptions"" table in ...",NaN,NaN
1,2,Maximal 3 Urlaubstage mit ins nächste Jahr neh...,HR,NaN,"Hallo Marcel, hier noch eine kurze Info für di...","Hello Marcel, here is a short info for you: Yo...",NaN,NaN
2,3,CR bedeutet Conversion Rate.,"Allgemeine Definitionen, CRM",NaN,"Hi Marcel, CR bedeutet in unserem Kontext Conv...","Hi Marcel, CR in our context means conversion ...",NaN,NaN
3,4,AE bedeutet Account Executive.,"Allgemeine Definitionen, CRM",NaN,"AE = Account Executive, also Sales Mitarbeiter...","AE = Account Executive, i.e. Sales employees w...",NaN,NaN
4,5,Mit “Abrechnungsdatum” ist das Attribut “Purch...,"Allgemeine Definitionen, DWH",NaN,"Hi Marcel, ja genau mit “Abrechnungsdatum” ist...","Hi Marcel, yes exactly with ""Billing Date"" the...",NaN,NaN
...,...,...,...,...,...,...,...,...
2695,27,Azure DevOps Lizenz. Wie war da der Prozess? ...,HR,NaN,"Hi Niklas, Stefan meinte, dass du für deinen P...","Hi Niklas, Stefan said that you had an Azure D...",26.0,538.0
2696,27,Azure DevOps Lizenz. Wie war da der Prozess? ...,HR,NaN,"Hi Niklas, Stefan meinte, dass du für deinen P...","Hi Niklas, Stefan said that you had got Enterp...",26.0,539.0
2697,27,Azure DevOps Lizenz. Wie war da der Prozess? ...,HR,NaN,"Hi Niklas, Stefan meinte, dass du für deinen P...","Hi Niklas, Stefan said before you had an Azure...",26.0,539.0
2698,27,Azure DevOps Lizenz. Wie war da der Prozess? ...,HR,NaN,"Hi Niklas, Stefan meinte, dass du für deinen P...","Hi Dan, Stefan said that you had our Enterpris...",26.0,539.0


In [149]:
sentences_synonym_swap_magic.to_excel('../data/demo_data_augmented.xlsx')